In [1]:
import sys
sys.path.append("..")

from dkt_dataset import Preprocess
from helper import train_base_lstm

In [2]:
import pandas as pd
from IPython.display import clear_output
from fe.feature import FEPipeline
from fe.agg import (
    AggFeBase,
    MakeCorrectCount, 
    MakeCorrectPercent, 
    MakeQuestionCount, 
    MakeTopNCorrectPercent
)

from fe.seq import (
    SplitAssessmentItemID,
    MakeFirstClass,
    MakeSecondClass,
    MakeTimeDiff,
    MakeYMD,
    ConvertTime
)

class MakeDifficultyByFirstClass(AggFeBase):
    name = "make_difficulty_by_firstclass"
    description = {
        "firstClassDifficulty" : "유저의 대분류별 정답률을 나타냅니다."
    }
    
    pre_fe = {"make_first_class"}
    agg_column = ["userID", "firstClass"]
    
    @classmethod
    def _transform(cls, df):
        grouped_df = df.groupby(cls.agg_column).answerCode.mean()
        a, b = zip(*grouped_df.index)
        
        right_df = pd.DataFrame(
            {"userID": a, "firstClass": b, "firstClassDifficulty": list(grouped_df)}
        )
        
        return right_df

In [36]:
from utils import get_args, get_root_dir
from helper import FeatureTestTrainer

In [20]:
class NewSplitPreprocess(Preprocess):
    def split_data(self):
        self.logger.info("Split based on User")
        self.logger.info(f"Original Train Dataset: {len(self.datas['train'])}")

        self.datas["valid"] = self.datas["test"][self.datas["test"].answerCode != -1]

        self.logger.info(f"Split Train Dataset: {len(self.datas['train'])}")
        self.logger.info(f"Split Valid Dataset: {len(self.datas['valid'])}")

In [21]:
args = get_args()
args.root_dir = get_root_dir("../split_test")
args.data_dir = "../../input/data/train_dataset/"

In [22]:
fe_pipeline = FEPipeline(
    args, [
        SplitAssessmentItemID,
        ConvertTime,
        MakeFirstClass,
        MakeSecondClass,
        MakeCorrectCount,
        MakeQuestionCount,
        MakeCorrectPercent,
        MakeDifficultyByFirstClass
    ]
)

columns = [
    "userID",
    "answerCode",
    "testPaper",
    "timeSec",
    "firstClass",
    "secondClass",
    "correctPer",
    "firstClassDifficulty"
]

pre_encoders = {
    "label": ["testPaper", "firstClass", "secondClass"],
    "min_max": ["correctPer"],
    "std": ["timeSec", "firstClassDifficulty"],
}

In [23]:
args.columns = columns[1:]

In [24]:
fe_pipeline.debug()
fe_pipeline.description()

[Feature Descriptions]

feature name : base_feature
feature type : seq
 - userID               : 사용자의 고유 번호입니다. 총 7,442명의 학생이 있습니다
 - assessmentItemID     : 사용자가 푼 문항의 일련 번호입니다.
 - testID               : 사용자가 푼 문항이 포함된 시험지의 일련 번호입니다.
 - answerCode           : 사용자가 푼 문항의 정답 여부를 담고 있는 이진 (0/1) 데이터입니다.
 - Timestamp            : 사용자가 문항을 푼 시간 정보입니다.
 - KnowledgeTag         : 사용자가 푼 문항의 고유 태그가 담겨져 있습니다.

feature name : split_assessmentitem_id
feature type : seq
 - testPaper            : 시험지 번호입니다.
 - testPaperCnt         : 시험지의 문항 번호입니다.

feature name : convert_time
feature type : seq
 - timeSec              : 사용자가 문항을 푼 타임스태프 정보입니다.

feature name : make_first_class
feature type : seq
 - firstClass           : 대분류에 해당합니다.

feature name : make_second_class
feature type : seq
 - secondClass          : 중분류에 해당합니다.

feature name : make_correct_count
feature type : agg
 - correctCnt           : 사용자가 맞춘 문항수를 나타냅니다.

feature name : make_question_count
feature type : agg
 - quesCnt              : 사

In [29]:
args.columns = ["userID"] + args.columns

In [66]:
preprocess = NewSplitPreprocess(args, fe_pipeline, columns)

In [67]:
preprocess.feature_engineering() # feature 
preprocess.split_data() # train => train, valid
preprocess.scaling(pre_encoders) # vector로 바꿔주고 Scaling해주고
preprocess.data_augmentation(choices=[3]) # [1, 2] 1: test_data추가, 2: user_id 기준 group_by 
clear_output()

In [68]:
args.columns

['answerCode',
 'testPaper',
 'timeSec',
 'firstClass',
 'secondClass',
 'correctPer',
 'firstClassDifficulty']

In [81]:
len(set(preprocess.get_data("train").userID.unique()).intersection(set(preprocess.get_data("test").userID.unique())))

0

In [82]:
len(set(preprocess.get_data("train").userID.unique()).intersection(set(preprocess.get_data("valid").userID.unique())))

0

In [157]:
train_data = preprocess.get_data("train_grouped")
valid_data = preprocess.get_data("valid_grouped")
test_data = preprocess.get_data("test_grouped")

In [74]:
from models.lstm.model import LSTM
trainer = FeatureTestTrainer(args, LSTM)

In [75]:
args.columns

['answerCode',
 'testPaper',
 'timeSec',
 'firstClass',
 'secondClass',
 'correctPer',
 'firstClassDifficulty']

In [76]:
train_loader, _ = trainer._get_loaders(train_data, valid_data)

In [77]:
trainer.debug(train_data, valid_data, test_data)

In [78]:
auc, acc = trainer.run_cv(train_data, valid_data, test_data, folds=5, seeds=[1, 2, 3, 4, 5])

wandb: wandb version 0.10.32 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade


Training steps: 0 Loss: 0.6914605498313904
Training steps: 50 Loss: 0.6795990467071533
Training steps: 100 Loss: 0.66401606798172
Training steps: 150 Loss: 0.6197633743286133
Training steps: 200 Loss: 0.611476480960846
VALID AUC : 0.7661739212593249 ACC : 0.7049138804457954



Training steps: 0 Loss: 0.6078208684921265
Training steps: 50 Loss: 0.6115068197250366
Training steps: 100 Loss: 0.5551007390022278
Training steps: 150 Loss: 0.5534874796867371
Training steps: 200 Loss: 0.5410237312316895
VALID AUC : 0.7719573638634197 ACC : 0.7033941236068896



Training steps: 0 Loss: 0.5134978890419006
Training steps: 50 Loss: 0.5707082748413086
Training steps: 100 Loss: 0.5677574276924133
Training steps: 150 Loss: 0.5889707803726196
Training steps: 200 Loss: 0.5838099718093872
VALID AUC : 0.7735396845936443 ACC : 0.7082066869300911



Training steps: 0 Loss: 0.5919654369354248
Training steps: 50 Loss: 0.5907580256462097
Training steps: 100 Loss: 0.5812669992446899
Training steps: 150 Loss: 0.5119457244873047
Training steps: 200 Loss: 0.5490837097167969
VALID AUC : 0.7739058032633344 ACC : 0.7084599797365755



...

Training steps: 0 Loss: 0.5586574077606201
Training steps: 50 Loss: 0.5942895412445068
Training steps: 100 Loss: 0.46654242277145386
Training steps: 150 Loss: 0.5214309096336365
Training steps: 200 Loss: 0.6107655167579651
VALID AUC : 0.780115560612813 ACC : 0.7094731509625126



Training steps: 0 Loss: 0.5862085819244385
Training steps: 50 Loss: 0.6036398410797119
Training steps: 100 Loss: 0.5881652235984802
Training steps: 150 Loss: 0.6017634272575378
Training steps: 200 Loss: 0.5775519609451294
VALID AUC : 0.7800508322855648 ACC : 0.7099797365754813



Training steps: 0 Loss: 0.5286556482315063
Training steps: 50 Loss: 0.5447274446487427
Training steps: 100 Loss: 0.5693047642707825
Training steps: 150 Loss: 0.5338605642318726
Training steps: 200 Loss: 0.673575758934021
VALID AUC : 0.7801695008855194 ACC : 0.709726443768997



Training steps: 0 Loss: 0.5401846170425415
Training steps: 50 Loss: 0.531859815120697
Training steps: 100 Loss: 0.5637170076370239
Training steps: 150 Loss: 0.5863024592399597
Training steps: 200 Loss: 0.554169774055481
VALID AUC : 0.7801730969036998 ACC : 0.7102330293819655



Training steps: 0 Loss: 0.49741095304489136
Training steps: 50 Loss: 0.49698102474212646
Training steps: 100 Loss: 0.6302911043167114
Training steps: 150 Loss: 0.5149060487747192
Training steps: 200 Loss: 0.5168042182922363
VALID AUC : 0.7801231379368361 ACC : 0.708966565349544



In [79]:
print(f"auc : {auc} acc : {acc}")
print(f"logging path : {trainer.prefix_save_path}")

auc : 0.7770310448109141 acc : 0.7075987841945289
logging path : ../split_test/LOG_[06.11_18:40]


**LB AUC : 0.7313 LB ACC : 0.6882**

In [158]:
auc, acc = trainer.run(train_data, valid_data, test_data)

wandb: wandb version 0.10.32 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade


Training steps: 0 Loss: 0.700603187084198
Training steps: 50 Loss: 0.6957963705062866
Training steps: 100 Loss: 0.6836996078491211
Training steps: 150 Loss: 0.6726133227348328
Training steps: 200 Loss: 0.665005624294281
Training steps: 250 Loss: 0.5939368009567261
VALID AUC : 0.77769436356305 ACC : 0.7060895822848515



Training steps: 0 Loss: 0.5544260740280151
Training steps: 50 Loss: 0.5366555452346802
Training steps: 100 Loss: 0.5577244758605957
Training steps: 150 Loss: 0.6209127306938171
Training steps: 200 Loss: 0.5739538669586182
Training steps: 250 Loss: 0.6146116256713867
VALID AUC : 0.783435254021612 ACC : 0.7101157523905385



Training steps: 0 Loss: 0.5501046776771545
Training steps: 50 Loss: 0.5619213581085205
Training steps: 100 Loss: 0.5576613545417786
Training steps: 150 Loss: 0.5835872888565063
Training steps: 200 Loss: 0.5580599308013916
Training steps: 250 Loss: 0.6445865631103516
VALID AUC : 0.7860476048638254 ACC : 0.7161550075490689



Training steps: 0 Loss: 0.5884982347488403
Training steps: 50 Loss: 0.519839882850647
Training steps: 100 Loss: 0.5290238261222839
Training steps: 150 Loss: 0.5839225053787231
Training steps: 200 Loss: 0.6324130892753601
Training steps: 250 Loss: 0.6108105182647705
VALID AUC : 0.7878165265510484 ACC : 0.7141419224962254



...

Training steps: 0 Loss: 0.5546300411224365
Training steps: 50 Loss: 0.7074905633926392
Training steps: 100 Loss: 0.6230628490447998
Training steps: 150 Loss: 0.7607958316802979
Training steps: 200 Loss: 0.4773101210594177
Training steps: 250 Loss: 0.5801773071289062
VALID AUC : 0.7901902857223907 ACC : 0.7166582788122798



Training steps: 0 Loss: 0.6786515712738037
Training steps: 50 Loss: 0.5602739453315735
Training steps: 100 Loss: 0.6185828447341919
Training steps: 150 Loss: 0.4971354007720947
Training steps: 200 Loss: 0.5733383893966675
Training steps: 250 Loss: 0.5385435223579407
VALID AUC : 0.7903868325765266 ACC : 0.7161550075490689



Training steps: 0 Loss: 0.5610966682434082
Training steps: 50 Loss: 0.5270990133285522
Training steps: 100 Loss: 0.6232728958129883
Training steps: 150 Loss: 0.6399747133255005
Training steps: 200 Loss: 0.5946403741836548
Training steps: 250 Loss: 0.5840374827384949
VALID AUC : 0.7902905854056868 ACC : 0.7166582788122798



Training steps: 0 Loss: 0.5220494866371155
Training steps: 50 Loss: 0.6842190027236938
Training steps: 100 Loss: 0.5202454328536987
Training steps: 150 Loss: 0.6269791126251221
Training steps: 200 Loss: 0.5264116525650024
Training steps: 250 Loss: 0.6108989119529724
VALID AUC : 0.7905924975836894 ACC : 0.7146451937594364



Training steps: 0 Loss: 0.5571156740188599
Training steps: 50 Loss: 0.6294474601745605
Training steps: 100 Loss: 0.5501905679702759
Training steps: 150 Loss: 0.5600674748420715
Training steps: 200 Loss: 0.600213885307312
Training steps: 250 Loss: 0.5868146419525146
VALID AUC : 0.7902601915622639 ACC : 0.7141419224962254



In [156]:
auc, acc

(0.7770310448109141, 0.7075987841945289)

In [84]:
preprocess.get_data("test")

,userID,answerCode,testPaper,timeSec,firstClass,secondClass,correctPer,firstClassDifficulty
0,3,1,804,-2.357753,4,267,0.69112,-0.005758
1,3,1,804,-2.357749,4,267,0.69112,-0.005758
2,3,0,804,-2.357736,4,266,0.69112,-0.005758
3,3,0,804,-2.357735,4,266,0.69112,-0.005758
4,3,0,804,-2.357734,4,265,0.69112,-0.005758
...,...,...,...,...,...,...,...,...
260109,7439,0,713,1.088795,3,783,0.62500,-0.135739
260110,7439,1,713,1.088798,3,783,0.62500,-0.135739
260111,7439,1,713,1.088801,3,761,0.62500,-0.135739
260112,7439,1,713,1.088813,3,761,0.62500,-0.135739


In [88]:
preprocess.get_data("valid").userID.nunique(), preprocess.get_data("test").userID.nunique()

(744, 744)

In [95]:
valid_grouepd = preprocess.get_data("valid_grouped")

In [97]:
import numpy as np

In [112]:
len(valid_data[0]) // 20

8

In [118]:
len(valid_data[0])

174

In [117]:
np.split(valid_data[0], 6)

[array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1]),
 array([1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1,
        1, 1, 1, 0, 1, 1, 1]),
 array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1,
        1, 1, 0, 1, 0, 1, 1]),
 array([1, 1, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1]),
 array([1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1]),
 array([1, 1, 1, 1, 0, 1, 0, 0, 0, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 0, 0, 1, 1, 0])]

In [121]:
np.split(valid_data[0])

ValueError: array split does not result in an equal division

In [103]:
len(valid_data)

7

In [123]:
max_seq_len = 20

In [149]:
new_valid_grouped = []

for valid_data in valid_grouepd:
    
    split_idx = (len(valid_data[0]) // max_seq_len) - 1
    
    if split_idx < 2:
        new_valid_grouped.append(valid_data)
        continue
    
    print(split_idx, end=' ')
    split_end = int(max_seq_len * split_idx)
    
    for valid_d in valid_data:
        split_d = np.split(valid_d[:split_end], split_idx)

7 42 5 4 20 29 10 23 14 25 19 15 23 15 9 15 26 31 2 4 30 37 6 17 16 15 16 18 27 10 23 15 8 16 28 18 11 7 4 11 28 25 38 17 8 18 19 29 8 8 3 9 14 19 32 4 26 17 18 8 4 19 32 5 4 7 5 15 19 19 3 10 16 7 12 9 11 5 30 23 18 16 4 14 4 3 15 42 13 5 30 8 6 26 10 22 14 2 2 17 12 25 5 2 3 24 16 11 4 6 6 6 21 2 9 12 2 8 28 8 30 10 7 24 9 5 26 23 15 10 5 3 5 7 7 23 12 3 22 13 10 9 17 9 11 13 18 5 5 22 27 24 25 6 2 14 11 15 17 11 16 26 20 16 27 17 13 8 14 6 4 9 33 19 5 17 10 12 17 15 4 19 29 6 25 13 12 10 9 14 25 9 9 28 4 5 4 24 4 12 22 29 29 12 14 7 6 20 13 27 15 21 14 19 24 10 17 4 21 22 8 12 12 11 5 5 16 13 8 3 7 37 7 30 3 2 8 11 9 22 17 16 10 9 10 24 19 18 23 9 33 16 27 14 11 2 18 12 10 2 8 23 7 17 7 9 11 15 2 2 26 17 11 26 9 16 5 22 7 7 11 2 21 10 6 21 14 13 9 14 9 6 9 8 5 16 20 5 6 7 10 3 14 43 21 15 28 10 4 7 8 9 8 10 3 10 24 24 22 17 9 19 4 12 3 19 10 21 6 17 30 17 23 11 3 30 16 18 9 2 23 10 30 8 11 14 17 10 18 11 16 17 42 5 6 22 6 8 10 20 7 14 7 10 11 6 14 6 20 5 16 15 15 6 21 9 20 8 3 11 4 

In [153]:
valid_data

(array([1, 1, 1, 1, 0, 1, 1, 1, 0, 0, 1, 0, 1, 1, 1]),
 array([586, 586, 586, 586, 586, 780, 780, 780, 780, 780, 780, 713, 713,
        713, 713]),
 array([-2.29728262, -2.29728162, -2.29727634, -2.29727448, -2.29726949,
         0.41494951,  0.41495108,  0.41495879,  0.41498305,  0.41498862,
         0.41498933,  1.08879517,  1.08879774,  1.08880074,  1.08881344]),
 array([3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3]),
 array([627, 627, 627, 627, 627, 237, 237, 237, 237, 237, 237, 783, 783,
        761, 761]),
 array([0.625, 0.625, 0.625, 0.625, 0.625, 0.625, 0.625, 0.625, 0.625,
        0.625, 0.625, 0.625, 0.625, 0.625, 0.625]),
 array([-0.13573852, -0.13573852, -0.13573852, -0.13573852, -0.13573852,
        -0.13573852, -0.13573852, -0.13573852, -0.13573852, -0.13573852,
        -0.13573852, -0.13573852, -0.13573852, -0.13573852, -0.13573852]))

In [151]:
np.array(new_valid_grouped)

<ipython-input-151-a1420e33b305>:1: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  np.array(new_valid_grouped)


array([[array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 1, 1, 1, 1, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1,
       0, 0, 0, 1, 1, 0, 0, 0]),
        array([363, 363, 363, 363, 363, 364, 364, 364, 364, 364, 372, 372, 372,
       372, 372, 373, 373, 373, 373, 373, 373, 373, 382, 382, 382, 382,
       382, 382, 382, 371, 371, 371, 371, 371, 371, 383, 383, 383, 383,
       383, 383, 383, 383, 384, 384, 384, 384, 385, 385, 385, 385, 385]),
        array([0.70953743, 0.70953971, 0.70954228, 0.70954727, 0.7095497 ,
       0.73347028, 0.73347556, 0.73347813, 0.7334837 , 0.73348627,
       0.7738529 , 0.77385732, 0.77386845, 0.77387159, 0.77387502,
       0.79563357, 0.795642  , 0.79564642, 0.79565712, 0.7956664 ,
       0.79566697, 0.79566768, 0.79619916, 0.79620987, 0.79621572,
       0.79621615, 0.79621643, 0.796217  , 0.79621757, 0.79759822,
       0.7976172 , 0.79763162, 0.79764903, 0.79765545, 0.79766359,
       0.80706581, 0.80706981, 0.80707

In [143]:
valid_data

(array([1, 1, 1, 1, 0, 1, 1, 1, 0, 0, 1, 0, 1, 1, 1]),
 array([586, 586, 586, 586, 586, 780, 780, 780, 780, 780, 780, 713, 713,
        713, 713]),
 array([-2.29728262, -2.29728162, -2.29727634, -2.29727448, -2.29726949,
         0.41494951,  0.41495108,  0.41495879,  0.41498305,  0.41498862,
         0.41498933,  1.08879517,  1.08879774,  1.08880074,  1.08881344]),
 array([3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3]),
 array([627, 627, 627, 627, 627, 237, 237, 237, 237, 237, 237, 783, 783,
        761, 761]),
 array([0.625, 0.625, 0.625, 0.625, 0.625, 0.625, 0.625, 0.625, 0.625,
        0.625, 0.625, 0.625, 0.625, 0.625, 0.625]),
 array([-0.13573852, -0.13573852, -0.13573852, -0.13573852, -0.13573852,
        -0.13573852, -0.13573852, -0.13573852, -0.13573852, -0.13573852,
        -0.13573852, -0.13573852, -0.13573852, -0.13573852, -0.13573852]))

In [131]:
valid_data

(array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 1, 1, 1, 1, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1,
        0, 0, 0, 1, 1, 0, 0, 0]),
 array([363, 363, 363, 363, 363, 364, 364, 364, 364, 364, 372, 372, 372,
        372, 372, 373, 373, 373, 373, 373, 373, 373, 382, 382, 382, 382,
        382, 382, 382, 371, 371, 371, 371, 371, 371, 383, 383, 383, 383,
        383, 383, 383, 383, 384, 384, 384, 384, 385, 385, 385, 385, 385]),
 array([0.70953743, 0.70953971, 0.70954228, 0.70954727, 0.7095497 ,
        0.73347028, 0.73347556, 0.73347813, 0.7334837 , 0.73348627,
        0.7738529 , 0.77385732, 0.77386845, 0.77387159, 0.77387502,
        0.79563357, 0.795642  , 0.79564642, 0.79565712, 0.7956664 ,
        0.79566697, 0.79566768, 0.79619916, 0.79620987, 0.79621572,
        0.79621615, 0.79621643, 0.796217  , 0.79621757, 0.79759822,
        0.7976172 , 0.79763162, 0.79764903, 0.79765545, 0.79766359,
        0.80706581, 0.80706981, 0.80707295, 0.80

In [94]:
model = trainer._get_model()

In [89]:
preprocess.get_data("valid")

,userID,answerCode,testPaper,timeSec,firstClass,secondClass,correctPer,firstClassDifficulty
0,3,1,804,-2.357753,4,267,0.69112,-0.005758
1,3,1,804,-2.357749,4,267,0.69112,-0.005758
2,3,0,804,-2.357736,4,266,0.69112,-0.005758
3,3,0,804,-2.357735,4,266,0.69112,-0.005758
4,3,0,804,-2.357734,4,265,0.69112,-0.005758
...,...,...,...,...,...,...,...,...
260108,7439,1,780,0.414989,3,237,0.62500,-0.135739
260109,7439,0,713,1.088795,3,783,0.62500,-0.135739
260110,7439,1,713,1.088798,3,783,0.62500,-0.135739
260111,7439,1,713,1.088801,3,761,0.62500,-0.135739


In [90]:
preprocess.get_data("test")

,userID,answerCode,testPaper,timeSec,firstClass,secondClass,correctPer,firstClassDifficulty
0,3,1,804,-2.357753,4,267,0.69112,-0.005758
1,3,1,804,-2.357749,4,267,0.69112,-0.005758
2,3,0,804,-2.357736,4,266,0.69112,-0.005758
3,3,0,804,-2.357735,4,266,0.69112,-0.005758
4,3,0,804,-2.357734,4,265,0.69112,-0.005758
...,...,...,...,...,...,...,...,...
260109,7439,0,713,1.088795,3,783,0.62500,-0.135739
260110,7439,1,713,1.088798,3,783,0.62500,-0.135739
260111,7439,1,713,1.088801,3,761,0.62500,-0.135739
260112,7439,1,713,1.088813,3,761,0.62500,-0.135739
